## OCHRE User Tutorial

* [Set up](#setup)
  * [Installation](#install)
  * [Getting input files](#inputs)
* [Running a basic house model](#dwelling)
* [Running a single equipment model](#equipment)
* [Running with external controllers](#control)
  * [Time-based HVAC setpoint control](#hvac-control)
  * [Occupancy-based setpoint control](#occupancy-control)

This tutorial can be downloaded online [here](https://github.com/NREL/OCHRE/blob/main/notebook/user_tutorial.ipynb).

Note that this tutorial only covers use cases with a single building or single piece of equipment.
OCHRE can be used to simulate a community or fleet of devices, both in parallel simulations or in co-simulation.


### <a name="setup"></a>Set up


#### <a name="install"></a>Installation

OCHRE can be installed using `pip` from the command line:

```
pip install git+https://github.com/NREL/OCHRE.get
```

We currently recommend using the `main` branch.

Note that OCHRE requires Python version 3.9 or higher.


#### <a name="inputs"></a>Getting input files

OCHRE Dwelling models require 3 inputs files:
* An HPXML file with building properties (xml)
* An occupancy schedule file (csv)
* A weather file (epw, or csv with NSRDB format)

We recommend using [ResStock](https://www.nrel.gov/buildings/resstock.html) (version 3.0+) or
[BEopt](https://www.nrel.gov/buildings/beopt.html) (version 3.0+) to create HPXML and occupancy schedule files.
OCHRE has sample files [here](https://github.com/NREL/OCHRE/tree/main/defaults/Input%20Files).

Weather files can be downloaded from [NSRDB](https://nsrdb.nrel.gov/),
[BEopt](https://www.nrel.gov/buildings/beopt.html),
or from ResStock's [dataset](https://data.nrel.gov/submissions/156).


### <a name="dwelling"></a>Running a basic house model

For more details, see [bin/run_dwelling.py](https://github.com/NREL/OCHRE/blob/main/bin/run_dwelling.py)

In [ ]:
import os
import datetime as dt
import pandas as pd

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files

simulation_name = 'Sample House'

dwelling_args = {
    # Timing parameters
    'start_time': dt.datetime(2018, 7, 1, 0, 0),  # year, month, day, hour, minute
    'time_res': dt.timedelta(minutes=10),         # time resolution of the simulation
    'duration': dt.timedelta(days=3),             # duration of the simulation

    # Input files
    'hpxml_file': os.path.join(default_input_path, 'Input Files', 'sample_resstock_properties.xml'),
    'schedule_input_file': os.path.join(default_input_path, 'Input Files', 'sample_resstock_schedule.csv'),
    'weather_file': os.path.join(default_input_path, 'Input Files', 'sample_resstock_weather.epw'),

    # Output parameters
    'verbosity': 4,                         # verbosity of time series files (0-9)
    'output_path': os.getcwd(),             # defaults to properties_file path

    # Equipment parameters (see bin/run_dwelling.py for more options)
    # 'Equipment': {
    #     'PV': {
    #         'capacity': 5,   # in kW
    #         'tilt': 20,      # in degrees
    #         'azimuth': 180,  # in degrees
    #     }
    # },
}

# Create Dwelling model
dwelling = Dwelling(name=simulation_name, **dwelling_args)


In [ ]:
# Run OCHRE simulation (returns DataFrames of timeseries results and a dictionary of metrics)
df_baseline, metrics, hourly = dwelling.simulate()

# Load results from previous run
# output_path = dwelling_args.get('output_path', os.path.dirname(dwelling_args['properties_file']))
# df, metrics, hourly = Analysis.load_ochre(output_path, simulation_name)

df_baseline.head()


In [ ]:
metrics

In [ ]:
%matplotlib

from ochre import CreateFigures

# Plot results
CreateFigures.plot_power_stack(df_baseline)
CreateFigures.plot_daily_profile(df_baseline, 'Total Electric Power (kW)', plot_max=False, plot_min=False)

### <a name="equipment"></a>Running a single equipment model

For more details and examples, see
[bin/run_equipment.py](https://github.com/NREL/OCHRE/blob/main/bin/run_equipment.py)

In [ ]:
import numpy as np

from ochre import ElectricResistanceWaterHeater

water_heater_args = {
    'start_time': dt.datetime(2018, 1, 1, 0, 0),  # year, month, day, hour, minute
    'time_res': dt.timedelta(minutes=1),
    'duration': dt.timedelta(days=10),
    'verbosity': 6,  # verbosity of results (1-9)
    'save_results': False,  # will not save results to a file
    
    # Equipment parameters
    'Initial Temperature (C)': 49,
    'Setpoint Temperature (C)': 51,
    'Deadband Temperature (C)': 5,
    'Capacity (W)': 4800,
    'Efficiency (-)': 1,
    'Tank Volume (L)': 250,
    'Tank Height (m)': 1.22,
    'UA (W/K)': 2.17,

    'schedule': None,  # defined below
}

# create example water draw schedule and add to equipment args
times = pd.date_range(water_heater_args['start_time'], water_heater_args['start_time'] + water_heater_args['duration'], 
                      freq=water_heater_args['time_res'])
water_draw_magnitude = 12  # L/min
hot_water_draws = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
water_heater_args['schedule'] = pd.DataFrame({
    'Showers (L/min)': hot_water_draws,
    'Zone Temperature (C)': 20,
    'Mains Temperature (C)': 7,
}, index=times)

# Initialize equipment
water_heater = ElectricResistanceWaterHeater(**water_heater_args)

In [ ]:
# Run simulation
df = water_heater.simulate()

# Show results
df.head()


In [ ]:
CreateFigures.plot_daily_profile(df, 'Water Heating Electric Power (kW)', plot_max=False, plot_min=False)
CreateFigures.plot_time_series_detailed((df['Hot Water Outlet Temperature (C)'],))


### <a name="control"></a>Running with external controllers

For more details and examples, see
[bin/run_external_control.py](https://github.com/NREL/OCHRE/blob/main/bin/run_external_control.py)

#### <a name="hvac-control"></a>Time-based HVAC setpoint control

This control will increase the cooling setpoint by 1C from 5-9PM each day.

We use the same house model as above.

In [ ]:
# Option 1: Adjust setpoints in the schedule, then run the simulation

# Create Dwelling model (same as above)
dwelling = Dwelling(name=simulation_name, **dwelling_args)

# Get HVAC heater setpoints
cooler = dwelling.get_equipment_by_end_use('HVAC Cooling')
setpoints = cooler.schedule['HVAC Cooling Setpoint (C)']

# Increase cooling setpoint by 1C from 5-9PM
peak_times = setpoints.between_time(dt.time(17, 0, 0), dt.time(21, 0, 0), inclusive='left').index
setpoints.loc[peak_times] += 1
cooler.reset_time()  # resets the schedule

# Run simulation
df, hourly, metrics = dwelling.simulate()

# Plot HVAC power and indoor temperature for baseline and control cases
data = {'Baseline': df_baseline,
        'Control': df}
plot_info = [('HVAC Cooling Electric Power (kW)', 'Cooling Power', 'r'),
             ('Temperature - Indoor (C)', 'Indoor Temperature', 'k', False)]
fig, (ax1, ax2) = CreateFigures.multi_comparison_plot(data, plot_info)
ax1.set_ylabel('Power (kW)')
ax2.set_ylabel('Temperature (C)')


In [ ]:
# Option 2: Adjust setpoints within the simulation

# Create Dwelling model
dwelling = Dwelling(name=simulation_name, **dwelling_args)

# Get HVAC heater and initial setpoint
cooler = dwelling.get_equipment_by_end_use('HVAC Cooling')

# Run OCHRE simulation in for loop
for t in dwelling.sim_times:
    assert dwelling.current_time == t

    # Get HVAC cooling setpoint, increase by 1C from 5-9PM
    setpoint = cooler.schedule.loc[t, 'HVAC Cooling Setpoint (C)']  # Original setpoint for current time
    if 17 <= t.hour <= 21:
        setpoint += 1

    # Set the external control signal
    control_signal = {
        'HVAC Cooling': {
            'Setpoint': setpoint,
        }
    }

    # Send the control signal to OCHRE and run for 1 time step
    house_status = dwelling.update(control_signal=control_signal)

# End the simulation
df, hourly, metrics = dwelling.finalize()

# Plot HVAC power and indoor temperature for baseline and control cases
data = {'Baseline': df_baseline,
        'Control': df}
plot_info = [('HVAC Cooling Electric Power (kW)', 'Cooling Power', 'r'),
             ('Temperature - Indoor (C)', 'Indoor Temperature', 'k', False)]
fig, (ax1, ax2) = CreateFigures.multi_comparison_plot(data, plot_info)
ax1.set_ylabel('Power (kW)')
ax2.set_ylabel('Temperature (C)')


#### <a name="occupancy-control"></a>Occupancy-based setpoint control

This control will increase the cooling setpoint by 1C if the house is unoccupied.

We use the same house model as above.

In [ ]:
# Create Dwelling model
dwelling = Dwelling(name=simulation_name, **dwelling_args)

# Get HVAC cooler and initial setpoint
cooler = dwelling.get_equipment_by_end_use('HVAC Cooling')
setpoint = cooler.schedule.loc[dwelling.start_time, 'HVAC Cooling Setpoint (C)']
occupied = True

# Run OCHRE simulation in for loop
for t in dwelling.sim_times:
    assert dwelling.current_time == t

    # Get HVAC heating setpoint and reduce by 1C if the house is unoccupied
    setpoint = cooler.schedule.loc[t, 'HVAC Cooling Setpoint (C)']  # Original setpoint for current time
    if not occupied:
        setpoint -= 1

    # Set the external control signal
    control_signal = {
        'HVAC Cooling': {
            'Setpoint': setpoint,
        }
    }

    # Send the control signal to OCHRE and run for 1 time step
    house_status = dwelling.update(control_signal=control_signal)

    # Update occupancy status
    occupied = house_status['Occupancy (Persons)'] > 0

# End the simulation
df, hourly, metrics = dwelling.finalize()

# Plot HVAC power and indoor temperature for baseline and control cases
data = {'Baseline': df_baseline,
        'Control': df}
plot_info = [('HVAC Cooling Electric Power (kW)', 'Cooling Power', 'r'),
             ('Temperature - Indoor (C)', 'Indoor Temperature', 'k', False),
             ('Occupancy (Persons)', 'Occupancy', 'b')]
fig, (ax1, ax2) = CreateFigures.multi_comparison_plot(data, plot_info)
ax1.set_ylabel('Power (kW) and Occupancy (Persons)')
ax2.set_ylabel('Temperature (C)')
